In [27]:
import os
from os import path
import json
import sys
from collections import defaultdict
from transformers import LongformerTokenizerFast

sys.path.append("/home/shtoshni/Research/fast-coref/src/")


In [28]:
input_file = "/home/shtoshni/Research/fast-coref/models/joint_downsample_30K/character_identification/test.log.jsonl"
base_output_dir = "/home/shtoshni/Research/fast-coref/models/html/coref"

model_name = path.basename(path.dirname(input_file))

output_dir = path.join(base_output_dir, "character_identification/joint")
if not path.exists(output_dir):
    os.makedirs(output_dir)
    
tokenizer = LongformerTokenizerFast.from_pretrained('allenai/longformer-large-4096', add_prefix_space=True)

SPEAKER_START = '[SPEAKER_START]'
SPEAKER_END = '[SPEAKER_END]'

tokenizer.add_special_tokens({
            'additional_special_tokens': [SPEAKER_START, SPEAKER_END]
        })

2

In [29]:
HTML_START = '<!DOCTYPE html><html lang="en"><head><meta charset="UTF-8"></head><body>'


start_tag_template = '<div style="border:2px; display:inline; border-style: {}; border-color: {}; padding: {}px; padding-right: 3px; padding-left: 3px">'
end_tag = '</div>'

largest_padding = 14
padding_reduction = 3


def get_tag_options(cluster_len, tag_type="gt"):
    if cluster_len > 1:
        border = 'solid'
    else:
        border = 'dotted'
    color = '#FFD700'
    if tag_type == "pred":
        color = 'red'
        
    return (border, color)

In [30]:

def get_mentions(clusters):
    mentions = set()
    mention_spans = set()
    for cluster_idx, cluster in enumerate(clusters):
        for mention in cluster:
            span_start, span_end = mention
            mentions.add((span_start, span_end))
            mention_spans.add((span_start, span_end))
            
    return mentions, mention_spans

In [31]:
html_files = []

with open(input_file) as f:
    for line in f:
        instance = json.loads(line.strip())

        bert_seg_idx = []
        doc_list = [] 
        for sentence in instance["sentences"]:
            doc_list.extend(sentence)
            bert_seg_idx.append(len(sentence) + (bert_seg_idx[-1] if len(bert_seg_idx) else 0))

        bert_seg_idx = set(bert_seg_idx)
        html_tag_list = {}

        # Get all the entity info
        clusters = sorted(instance["clusters"], key=lambda cluster: min([elem[0] for elem in cluster]))
        pred_clusters = sorted(instance["predicted_clusters"], key=lambda cluster: min([elem[0] for elem in cluster]))
        
        all_mentions, all_spans = get_mentions(clusters)
        all_pred_mentions, all_pred_spans = get_mentions(pred_clusters)
        
        for cluster_idx, cluster in enumerate(clusters):
            for mention in cluster:
                span_start, span_end = mention
                
                corr = '' 
                if (span_start, span_end) not in all_pred_mentions:
                    corr = 'MISSED '
                    if (span_start, span_end) in all_pred_spans:
                        corr = ''

                span_end = span_end + 1  ## Now span_end is not part of the span
                ent_type = f'{corr}' #, {mention_info["realis"]}'
                       

                if span_start not in html_tag_list:
                    html_tag_list[span_start] = defaultdict(list)
                if span_end not in html_tag_list:
                    html_tag_list[span_end] = defaultdict(list)

                subscript = ent_type + " " + str(cluster_idx)

                tag_options = get_tag_options(len(cluster))
                start_tag = start_tag_template.format(
                    *tag_options, 
                    largest_padding - padding_reduction * len(html_tag_list[span_start]['start']))


                html_tag_list[span_start]['start'].append((start_tag, span_end))
                # Subscript used in end
                html_tag_list[span_end]['end'].append((span_start, cluster_idx, end_tag, subscript))
                
        
        # Get all the entity info
        
        for cluster_idx, cluster in enumerate(pred_clusters):
            for mention in cluster:
                span_start, span_end = mention
                corr = ''
                if (span_start, span_end) not in all_mentions:
                    corr = 'WRONG '
                    if (span_start, span_end) not in all_spans:
                        corr = 'EXTRA '
                        
                
                span_end = span_end + 1  ## Now span_end is not part of the span
                
                ent_type = f'{corr}'

                if span_start not in html_tag_list:
                    html_tag_list[span_start] = defaultdict(list)
                if span_end not in html_tag_list:
                    html_tag_list[span_end] = defaultdict(list)

                subscript = ent_type + " " + str(cluster_idx)

                tag_options = get_tag_options(len(cluster), "pred")
                start_tag = start_tag_template.format(
                    *tag_options, 
                    largest_padding - padding_reduction * len(html_tag_list[span_start]['start']))


                html_tag_list[span_start]['start'].append((start_tag, span_end))
                # Subscript used in end
                html_tag_list[span_end]['end'].append((span_start, cluster_idx, end_tag, subscript))


        html_string = HTML_START + '<div style="line-height: 3">'
        for token_idx, token in enumerate(doc_list):
            
            if token_idx in bert_seg_idx:
                html_string += "\n<br/>"

            if token_idx in html_tag_list:
                for tag_type in ['end', 'start']:
                    if tag_type == 'end' and (tag_type in html_tag_list[token_idx]):
                        tags = html_tag_list[token_idx]['end']

                        # Sort the tags so as to mimic the stack behavior
                        tags = tags[::-1]  # Highest mentions first
                        for _, _, html_tag, subscript in tags:
                            html_string += "<sub>" + subscript + "</sub>" 
                            html_string += html_tag
                            # Since we are deleting the highest indices first, the lower indices are unaffected

                    if tag_type == 'start' and (tag_type in html_tag_list[token_idx]):
                        sorted_tag_list = sorted(html_tag_list[token_idx]['start'], key=lambda x: x[1], reverse=True)
                        for html_tag, _ in sorted_tag_list:
                            html_string += html_tag
            
            token_string = tokenizer.convert_ids_to_tokens(token).strip()
            if token_string[0] == "Ġ":
                token_string = token_string[1:]
            html_string += " " + token_string

        html_string += "</div></body></html>"
        html_string = html_string.replace("\n", "\n<br/>")
        html_string = html_string.replace("~", "&lt;")
        html_string = html_string.replace("^", "&gt;")

        file_name = instance["doc_key"].replace("/", "-") + ".html"
#         print(file_name)
        file_path = path.join(output_dir, file_name)
        html_files.append(file_name)
        with open(file_path, "w") as f:
            f.write(html_string)


In [32]:
index_html = HTML_START + 'Tag usage:<br/> <ul type="1">'

index_html += '<li> Ground truth in Golden, and Pred mentions in Pink rectangles </li>\n' 
index_html += '<li> Singletons in dotted, and chains in solid borders </li>'
index_html += '''<li> To highlight errors, there are a few text markers: 
(a) MISSED - mention in GT not in pred, (b) EXTRA - mention in pred not in GT, (c) WRONG - span match but wrong mention type</li>'''

index_html += '</ul>\n\n<ol type="1">'



for html_file in html_files:
    base_name = path.splitext(path.basename(html_file))[0].replace("-", "/")
    index_html += '<li> <a href="{}", target="_blank">'.format(html_file) + base_name + '</a></li>\n'
    
index_html += '</ol>\n</body>\n</html>'
index_file_path = path.join(output_dir, "index.html")
print(index_file_path)
with open(path.join(output_dir, "index.html"), "w") as g:
    g.write(index_html)

/home/shtoshni/Research/fast-coref/models/html/coref/character_identification/joint/index.html
